In [1]:
from solve_LMI import solve_LMI
from solve_LP import solve_LP
import numpy as np
from scipy.linalg import toeplitz
from circulant_embedding import faster_A
from scipy.sparse import diags
from SCGAL import run_solver
import time
import json

In [2]:
n = 1
max_runs = 100

starting_d = 16
ending_d = 20
ds = list(range(starting_d, ending_d+1))
circ_emb_obj = np.zeros((max_runs, len(ds)))
circ_emb_time = np.zeros((max_runs, len(ds)))
LMI_obj = np.zeros((max_runs, len(ds)))
LMI_time = np.zeros((max_runs, len(ds)))
SCGAL_obj = np.zeros((max_runs, len(ds)))
SCGAL_time = np.zeros((max_runs, len(ds)))
cs = np.random.normal(size=(4*ds[-1], len(ds)))

# d = ds[0]
# c = cs[:4*d, 0]

In [3]:
# Circulant embedding
for i in range(len(ds)):
    d = ds[i]
    c = cs[:4*d, i]

    A = faster_A(n, d)
    A_0 = np.ones(np.shape(A)[0])

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        tic = time.perf_counter()
        status_LP, optimal_value_LP, optimal_vars_LP = solve_LP(A_0, A, c, verbose=False)
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        circ_emb_obj[num_iterations, i] = optimal_value_LP
        circ_emb_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)
        
        # print("LP Status:", status_LP)
        # print("LP Optimal Value:", optimal_value_LP)
        # print("LP Optimal Variables:", [var.value for var in optimal_vars_LP])

16 0.010504662001039833
16 0.01712439104449004
16 0.023092605988495052
16 0.02879297296749428
16 0.034150872961618006
16 0.03966733597917482
16 0.045175880019087344
16 0.050613849016372114
16 0.05626794300042093
16 0.06164703698595986
16 0.06701262196293101
16 0.07242767995921895
16 0.07786839595064521
16 0.08320262795314193
16 0.08871064195409417
16 0.09403539495542645
16 0.09931974794017151
16 0.10468733095331118
16 0.11017291195457801
16 0.11550503695616499
16 0.120931037934497
16 0.12652603595051914
16 0.13182215695269406
16 0.13717385294148698
16 0.1425310299382545
16 0.14780724793672562
16 0.1531030159094371
16 0.1585045249084942
16 0.16377078386722133
16 0.16901377990143374
16 0.1742482119007036
16 0.1795643928926438
16 0.18492718890774995
16 0.19014151889132336
16 0.1954770479351282
16 0.20069284789497033
16 0.20593745989026502
16 0.2111951668630354
16 0.21765585587127134
16 0.22355568589409813
16 0.22926549793919548
16 0.23508122994098812
16 0.24083187995711342
16 0.2464351749

In [4]:
# LMI formulation with toeplitz constraints
for i in range(len(ds)):
    d = ds[i]
    c = cs[:4*d, i]

    col = np.zeros(2*d + 1, dtype=complex)
    col[0] = 1
    A_0 = toeplitz(col)

    A = []

    for m in range(1, 2*d + 1):
        col = np.zeros(2*d + 1, dtype=complex)
        col[m] = 1
        mx = toeplitz(col)
        A.append(mx)

    for m in range(1, 2*d + 1):
        col = np.zeros(2*d + 1, dtype=complex)
        col[m] = -1j
        mx = toeplitz(col)
        A.append(mx)

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        tic = time.perf_counter()
        status_LMI, optimal_value_LMI, optimal_vars_LMI = solve_LMI(A_0, A, c, verbose=False)
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        LMI_obj[num_iterations, i] = optimal_value_LMI
        LMI_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)

    # print("Toeplitz LMI Status:", status_LMI)
    # print("Toeplitz LMI Optimal Value:", optimal_value_LMI)
    # print("Toeplitz LMI Optimal Variables:", [var.value for var in optimal_vars_LMI])

16 0.5461484330007806
16 1.1210508949588984
16 1.6611763020046055
16 2.2043030209606513
16 2.741531364968978
16 3.2808924759738147
16 3.8201489149942063
16 4.382544449006673
16 4.92616493901005
16 5.49093318299856
16 6.030361612967681
16 6.5679265419603325
16 7.1069269310100935
16 7.643584896984976
16 8.182741016964428
16 8.723185295937583
16 9.273688954941463
16 9.811897939885966
16 10.35138326487504
16 10.891703859902918
16 11.440806549915578
16 11.978380423912313
16 12.522752826916985
16 13.104850119969342
16 13.659944059967529
16 14.198964393988717
16 14.740025254024658
16 15.285893253050745
16 15.829320851014927
16 16.370479086996056
16 16.914863691024948
16 17.45095821801806
16 17.98860839701956
16 18.526628925057594
16 19.07410226808861
16 19.613212283118628
16 20.153440066089388
16 20.690386766102165
16 21.227464603143744
16 21.767265925183892
16 22.303662604186684
16 22.84164824121399
16 23.37849604722578
16 23.914215725206304
16 24.453275635198224
16 25.027488643187098
16 25.

In [5]:
# SPARSE LMI formulation for Sketchy CGAL
for i in range(len(ds)):
    d = ds[i]
    c = cs[:4*d, i]

    diag = np.ones(2*d + 1, dtype=complex)
    A_0 = diags(diag)

    A = []

    for m in range(1, 2*d + 1):
        diag = np.ones(2*d + 1 - m, dtype=complex)
        mx = diags([diag, diag],offsets=[-m,m])
        A.append(mx)

    for m in range(1, 2*d + 1):
        diag = np.ones(2*d + 1 - m, dtype=complex)
        mx = diags([-1j*diag, 1j*diag],offsets=[-m,m])
        A.append(mx)

    A_norm = 2*d

    start = time.perf_counter()
    total_time = 0
    num_iterations = 0
    while total_time < 60 and num_iterations < 100:
        tic = time.perf_counter()
        U, Lambda, objective, Omega, z, y, S = run_solver(A_0, A, c, 2*d + 1, 4*d,
                                                        alpha=2*np.linalg.norm(c,ord=2), A_norm=A_norm,
                                                        R=1, T=5000,
                                                        trace_mode='bound')
        toc = time.perf_counter()
        time_of_this_iter = toc - tic
        total_time += time_of_this_iter
        print(d, total_time)
        SCGAL_obj[num_iterations, i] = -objective
        SCGAL_time[num_iterations, i] = time_of_this_iter
        num_iterations += 1
    print(num_iterations)

    # print("Calculated Objective:", -objective)

16 9.393264804035425
16 17.59962458803784
16 24.138501751061995
16 31.859762195090298
16 38.05702348705381
16 44.15016857808223
16 52.69764107710216
16 58.4609598350944
16 66.13742015004391
9
17 59.038293203979265
17 118.10661279194755
2


ArpackNoConvergence: ARPACK error -1: No convergence (311 iterations, 0/1 eigenvectors converged)

In [ ]:
my_dict = {'ds': list(ds), 'circ_emb_obj': circ_emb_obj.tolist(), 'circ_emb_time': circ_emb_time.tolist(),
           'LMI_obj': LMI_obj.tolist(), 'LMI_time': LMI_time.tolist(),
           'SCGAL_obj': SCGAL_obj.tolist(), 'SCGAL_time': SCGAL_time.tolist()}
# Save the results to a JSON file
file_name = 'ds' + str(ds[0]) + 'to' + str(ds[-1]) + '.json'
with open(file_name, 'w') as f:
    json.dump(my_dict, f)

In [ ]:
with open(file_name, 'r') as f:
    loaded_dict = json.load(f)

In [ ]:
ds_loaded = np.array(loaded_dict['ds'])
circ_emb_obj_loaded = np.array(loaded_dict['circ_emb_obj'])
circ_emb_time_loaded = np.array(loaded_dict['circ_emb_time'])
LMI_obj_loaded = np.array(loaded_dict['LMI_obj'])
LMI_time_loaded = np.array(loaded_dict['LMI_time'])
SCGAL_obj_loaded = np.array(loaded_dict['SCGAL_obj'])
SCGAL_time_loaded = np.array(loaded_dict['SCGAL_time'])

In [ ]:
assert np.allclose(ds, ds_loaded)
assert np.allclose(circ_emb_obj, circ_emb_obj_loaded)
assert np.allclose(circ_emb_time, circ_emb_time_loaded)
assert np.allclose(LMI_obj, LMI_obj_loaded)
assert np.allclose(LMI_time, LMI_time_loaded)
assert np.allclose(SCGAL_obj, SCGAL_obj_loaded)
assert np.allclose(SCGAL_time, SCGAL_time_loaded)

In [ ]:
print(SCGAL_obj)